This notebook has been made in colab research. To run import into colab and convert bash cells (in markdown) to code cells to download and manage dataset.

```bash
!pip install -q kaggle

!mkdir ~/.kaggle
!cp -f /root/kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d nadyana/flowers
```

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/flowers.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

```bash
!rm -rf flowers/lotus; rm -rf flowers/bellflower
```

In [ ]:
from pathlib import Path
import imghdr
import os

data_dir = "flowers/"
image_extensions = [".png", ".jpg"]  # add there all your images file extensions

img_type_accepted_by_tf = ["bmp", "gif", "jpeg", "png"]
for filepath in Path(data_dir).rglob("*"):
    if filepath.suffix.lower() in image_extensions:
        img_type = imghdr.what(filepath)
        if img_type is None:
            print(f"{filepath} is not an image")
            os.remove(filepath)
        elif img_type not in img_type_accepted_by_tf:
            print(f"{filepath} is a {img_type}, not accepted by TensorFlow")
            os.remove(filepath)

In [ ]:
BATCH_SIZE = 32
IMAGE_SIZE = (128,128)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout, Activation
from keras.optimizers import SGD

In [ ]:
train_ds, val_ds = keras.utils.image_dataset_from_directory(
    directory   = '/content/flowers',
    validation_split = 0.15,
    seed        = 1111,
    subset      = "both",
    labels      ='inferred',
    label_mode  = 'int',
    batch_size  = BATCH_SIZE,
    image_size  = IMAGE_SIZE
)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

In [ ]:
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_ds = train_ds.map(process)
val_ds = val_ds.map(process)

In [ ]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (128,128,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
 
model.add(Conv2D(filters =96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters = 96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(256, activation = "relu"))
model.add(Dense(5, activation = "softmax"))

In [ ]:
model.summary()

In [ ]:
from keras.optimizers import SGD,Adam
learn_rate=.001

# https://stackoverflow.com/questions/49922252/choosing-number-of-steps-per-epoch
batch_size=150
epochs=4

# https://keras.io/api/optimizers/sgd/
sgd = SGD(learning_rate=learn_rate, momentum=.9, nesterov=True)
model.compile(optimizer=sgd,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#from keras.callbacks import ReduceLROnPlateau
#red_lr= ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.1)
#model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
EvMod = model.fit(train_ds, epochs=10, validation_data=val_ds)

In [ ]:
model.save("./test-v3.h5")

In [ ]:
loaded_model = tf.keras.saving.load_model("./test-v3.h5")

In [ ]:
import cv2

image_size = (128, 128)

imagem = cv2.imread('/content/campo-com-girassol-683x1024.jpg')
#imagem = cv2.imread('/content/girassol_04jul22_Mahathir-Mohd-Yasin_EyeEm_Gettyimages-768x512.jpg')
imagem = cv2.resize(imagem, image_size)
imagem = imagem.reshape((1,128,128,3))
imagem = tf.cast(imagem/255. ,tf.float32)

model.predict(imagem).round()